In [2]:
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_id = "rajistics/informal_formal_style_transfer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def informal_to_formal(text, max_new_tokens=64, num_beams=4):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=2,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# test
print(informal_to_formal("gimme that report now"))
print(informal_to_formal("i loooooooooooooooooooooooove going to the movies."))


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

I am going to get that report now.
I love going to the movies.


In [3]:
def informal_to_formal_prefixed(text, **gen_kwargs):
    prefixed = "transfer Casual to Formal: " + text
    return informal_to_formal(prefixed, **gen_kwargs)


In [4]:
import gradio as gr

def formal_interface(text, max_len, beams):
    return informal_to_formal(text, max_new_tokens=int(max_len), num_beams=int(beams))

demo = gr.Interface(
    fn=formal_interface,
    inputs=[
        gr.Textbox(lines=3, label="Informal text"),
        gr.Slider(16, 128, value=64, step=4, label="Max new tokens"),
        gr.Slider(1, 8, value=4, step=1, label="Beams"),
    ],
    outputs=gr.Textbox(label="Formal text"),
    title="Informal ➜ Formal ",
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://591bd78c0ee0426622.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
model.save_pretrained("my_formal_t5_model")
print("Model saved to: my_formal_t5_model")

Model saved to: my_formal_t5_model


In [ ]:
# Alternative 1: Save only model weights (state_dict)
import torch
torch.save(model.state_dict(), "formal_model_weights.pth")
print("Model weights saved to: formal_model_weights.pth")

# To load later:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
# model.load_state_dict(torch.load("formal_model_weights.pth"))
# model.to(device)

In [ ]:
# Alternative 2: Save in SafeTensors format (more secure and faster loading)
try:
    from safetensors.torch import save_file
    save_file(model.state_dict(), "formal_model_weights.safetensors")
    print("Model saved in SafeTensors format: formal_model_weights.safetensors")
except ImportError:
    print("SafeTensors not installed. Install with: pip install safetensors")

# To load SafeTensors:
# from safetensors.torch import load_file
# state_dict = load_file("formal_model_weights.safetensors")
# model.load_state_dict(state_dict)

In [ ]:
# Alternative 3: Save model and tokenizer to a custom directory
model.save_pretrained("./my_custom_formal_model")
tokenizer.save_pretrained("./my_custom_formal_model")
print("Model and tokenizer saved to: ./my_custom_formal_model/")

# Alternative 4: Push to Hugging Face Hub (requires huggingface_hub)
# from huggingface_hub import login
# login()  # You'll need to authenticate
# model.push_to_hub("your-username/formal-style-transfer-model")
# tokenizer.push_to_hub("your-username/formal-style-transfer-model")
# print("Model pushed to Hugging Face Hub")